In [2]:
#import necessary packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import tree

from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import time

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
trlabels = pd.read_csv("train_labels.csv")
trlabels = reduce_memory_usage(trlabels)
sample = pd.read_csv("sample_submission.csv")

In [58]:
trlabels['session'] = trlabels.session_id.apply(lambda s: int(s.split('_')[0]))
trlabels['question'] = trlabels.session_id.apply(lambda s: int(s.split('_')[1][1:]))

In [53]:
#split train into 3 groups based on their levels
df_0_4 = train[train['level_group'] == '0-4']
df_0_4 = df_0_4[['level','session_id', 'index', 'elapsed_time', 'screen_coor_x']]

df_5_12 = train[train['level_group'] == '5-12']
df_5_12 = df_5_12[['level','session_id', 'index', 'elapsed_time', 'screen_coor_x']]

df_13_22 = train[train['level_group'] == '13-22']
df_13_22 = df_13_22[['level','session_id', 'index', 'elapsed_time', 'screen_coor_x']]

In [6]:
# def delt_time_def(df):
#     df.sort_values(by=['session_id', 'elapsed_time'], inplace=True)
#     df['delt_time'] = df['elapsed_time'].diff(1) # 对于同一个玩家的触发两个相邻的事件时的时间差
#     df['delt_time'].fillna(0, inplace=True)
#     df['delt_time'].clip(0, 103000, inplace=True) # 为什么是103000 s = 1716 min = 28 h
#     return df

In [74]:
# Categorical features 
CATS = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']

# Numerical features
NUMS = ['elapsed_time','level','page','room_coor_x', 'room_coor_y', 
        'screen_coor_x', 'screen_coor_y', 'hover_duration']

# define the function to create the features
def feature_engineer(train):
    dfs = []
    for c in CATS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('mean')
        dfs.append(tmp)
    for c in NUMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    df = pd.concat(dfs,axis=1)
    df = df.fillna(-1)
    df = df.reset_index()
    df = df.set_index('session_id')
    return df

In [75]:
new_train = feature_engineer(train)
FEATURES = [c for c in new_train.columns if c != 'level_group'] # all features except the target
print('We will train with', len(FEATURES) ,'features')
ALL_USERS = new_train.index.unique() # treat each index as a user
print('We will train with', len(ALL_USERS) ,'users info')

We will train with 21 features
We will train with 23562 users info


In [81]:
clf = LinearRegression()
oof = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)
models = {}

In [ ]:
 # ITERATE THRU QUESTIONS 1 THRU 18
for t in range(1,19):
        print(t,', ',end='')
        
        # USE THIS TRAIN DATA WITH THESE QUESTIONS
        if t<=3: grp = '0-4'
        elif t<=13: grp = '5-12'
        elif t<=22: grp = '13-22'
            
        # TRAIN DATA
        train_x = train.iloc[train_index]
        train_x = train_x.loc[train_x.level_group == grp]
        train_users = train_x.index.values
        train_y = targets.loc[targets.q==t].set_index('session').loc[train_users]
        
        # VALID DATA
        valid_x = train.iloc[test_index]
        valid_x = valid_x.loc[valid_x.level_group == grp]
        valid_users = valid_x.index.values
        valid_y = targets.loc[targets.q==t].set_index('session').loc[valid_users]
        
        # TRAIN MODEL
#         clf = CatBoostClassifier(n_estimators=500, depth=4, learning_rate=0.01, random_state=0) 
        clf.fit(train_x[FEATURES].astype('float32'), train_y['correct'])
        
        # SAVE MODEL, PREDICT VALID OOF
        models[f'{grp}_{t}'] = clf
#         oof.loc[valid_users, t-1] = clf.predict(valid_x[FEATURES].astype('float32'))[:,1]
        
        print()

In [85]:
from sklearn.model_selection import KFold, GroupKFold
gkf = GroupKFold(n_splits=10)
oof = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)
models = {}

# COMPUTE CV SCORE WITH 5 GROUP K FOLD
for i, (train_index, test_index) in enumerate(gkf.split(X=train, groups=train.index)):
    print('#'*25)
    print('### Fold',i+1)
    print('#'*25)
    
    # ITERATE THRU QUESTIONS 1 THRU 18
    for t in range(1,19):
        print(t,', ',end='')
        
        # USE THIS TRAIN DATA WITH THESE QUESTIONS
        if t<=3: grp = '0-4'
        elif t<=13: grp = '5-12'
        elif t<=22: grp = '13-22'
            
        # TRAIN DATA
        train_x = train.iloc[train_index]
        train_x = train_x.loc[train_x.level_group == grp]
        train_users = train_x.index.values
        train_y = trlabels.loc[trlabels.question==t].set_index('session').loc[train_users]
        
        # VALID DATA
        valid_x = train.iloc[test_index]
        valid_x = valid_x.loc[valid_x.level_group == grp]
        valid_users = valid_x.index.values
        valid_y = trlabels.loc[trlabels.question ==t].set_index('session').loc[valid_users]
        
        # TRAIN MODEL
        clf = LinearRegression() 
        clf.fit(train_x[FEATURES].astype('float32'), train_y['correct'])
        
        # SAVE MODEL, PREDICT VALID OOF
        models[f'{grp}_{t}'] = clf
        oof.loc[valid_users, t-1] = clf.predict(valid_x[FEATURES].astype('float32'))[:,1]
        
    print()

#########################
### Fold 1
#########################
1 , 

KeyError: "None of [Int64Index([       0,        1,        2,        3,        4,        6,\n                   7,        8,        9,       10,\n            ...\n            26295599, 26295600, 26295601, 26295602, 26295604, 26295605,\n            26295606, 26295607, 26295608, 26295609],\n           dtype='int64', name='session', length=3583080)] are in the [index]"

In [55]:
kol_lvl = (df_0_4.groupby(['session_id'])['level'].agg('nunique') < 5) 
# 统计每组的level中的值的种类数量，将种类数小于5的组标记为True，也就是只要level数小于5说明这一阶段的都没有通关
# print(kol_lvl.shape)
# print(kol_lvl)
list_session = kol_lvl[kol_lvl].index  
# 又因为kol_lvl的行索引就是session_id，因此直接将筛选的布尔矩阵的放进kol_lvl就可以，选择正确的id
# print(kol_lvl[kol_lvl])
df_0_4 = df_0_4[~ df_0_4['session_id'].isin(list_session) ] 
# session_id的值不在这个list_session就为False，否则为True，然后取反：获得符合要求的id

# df_0_4 = delt_time_def(df_0_4)
train1 = df_0_4

print(f"train1 shape : {train1.shape}")

kol_lvl = (df_5_12.groupby(['session_id'])['level'].agg('nunique') < 8) # 小于8个的id没有通关，不要
list_session = kol_lvl[kol_lvl].index  # index返回的是不符合要求的session_id，因为行索引就是session_id

df_5_12 = df_5_12[~df_5_12['session_id'].isin(list_session)] # 如果某一列是不正确的id，则取反变成False不选择这一行

# df_5_12 = delt_time_def(df_5_12)
train2 = df_5_12

print(f"train2 shape : {train2.shape}")

kol_lvl = (df_13_22.groupby(['session_id'])['level'].agg('nunique') < 10) # 小于10个的id没有通关，不要
list_session = kol_lvl[kol_lvl].index
df_13_22 = df_13_22[~ df_13_22['session_id'].isin(list_session) ]

# df_13_22 = delt_time_def(df_13_22)
train3 = df_13_22

print(f"train3 shape : {train3.shape}")

train1 shape : (3981005, 5)
train2 shape : (8844030, 5)
train3 shape : (13123678, 5)


In [56]:
FEATURES1 = [c for c in train1.columns if c not in ['level_group']] # 由于这里只有NUM类型的数据因此每个level的
FEATURES2 = [c for c in train2.columns if c not in ['level_group']]
FEATURES3 = [c for c in train3.columns if c not in ['level_group']]
print('We will train with', len(FEATURES1), len(FEATURES2), len(FEATURES3) ,'features')
ALL_USERS = train1.index.unique()
print('We will train with', len(ALL_USERS) ,'users info')

We will train with 5 5 5 features
We will train with 3981005 users info


In [30]:
# xgb_params = {
#     'objective' : 'binary:logistic',
#     'eval_metric':'logloss',
#     'learning_rate': 0.05,
#     'max_depth': 4,
#     'n_estimators': 1000,
#     'early_stopping_rounds': 50,
#     'tree_method':'hist',
#     'subsample':0.8,
#     'colsample_bytree': 0.4,
#     'use_label_encoder' : False}
models = {}

In [ ]:
for t in range(1,19):
    # 分别训练不同问题的分类器
    if t <= 3:
        grp = '0-4'
        df = train1.fillna(-1)
        FEATURES = FEATURES1
        
    elif t <= 13:
        grp = '5-12'
        df = train2.fillna(-1)
        FEATURES = FEATURES2
    
    elif t <= 22:
        grp = '13-22'
        df = train3.fillna(-1)
        FEATURES = FEATURES3
    
    # TRAIN DATA
    train_x = df
#     print(train_x.shape, end=',')
    train_users = train_x.index.values # 返回train数据集中存在的session_id的列表
    #print(train_users)
    print(trlabels.loc[trlabels['question'] == t].set_index('session').loc[train_users])
    print()
    print(trlabels.loc[trlabels['question'] == t].set_index('session'))
#     print()
#     print(trlabels.loc[trlabels['question'] == t].set_index('session').loc[0])
#     print(train_users, end)
    #train_y = trlabels.loc[trlabels['question'] == t].set_index('session').loc[train_users] # 选择第t个问题的标签且id在训练数据中的
    #train_y = train_y['correct']
#     print(train_y.shape)

# #     print("Xtrain")
# #     print(X_train[FEATURES])
#     X_train, X_test, y_train, y_test = train_test_split(train_x[['l_ev_name_observation_click','t_ev_name_observation_click']], train_y, test_size=0.2)
#     clf = LinearRegression()
#     clf.fit(X_train.astype('float32'), y_train)
    
# #     print(X_train[['l_ev_name_observation_click','t_ev_name_observation_click']])
# #     print(f'quiz{t}, train data shape is {X_train[FEATURES].shape}, test data shape is {y_train.shape}')
#     print("train data F1 score is ", f1_score(y_train, clf.predict(X_train)))
# #     print(f'test data F1 score is ', f1_score(y_test, clf.predict(X_test)))
#     # SAVE MODEL, PREDICT VALID OOF
#     models[f'{grp}_{t}'] = clf

In [ ]:
print(trlabels['session_id'].values)
print(trlabels['Question'].values)

# print(trlabels['IDQ'].iloc[i].index('_'))
#separate session ids into question groups (1-18)
#merge train w/ train_labels to get correct column

In [ ]:
#merge trlabels and train to get Y (correct) and then predict 

In [ ]:
group1 = trlabels.groupby('Question').get_group('q1')

IDs = trlabels.groupby('Question').get_group('q1')['session_id']

In [ ]:
g1 = train[~train['session_id'].isin(IDs)]  

In [ ]:
t = g1.groupby('level_group').get_group('0-4')
# print(len(g1['session_id']))

# print(len(group1))

In [ ]:
#create train and test data 
X = g1
Y = group1
print(len(X))
print(len(Y))
# train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.4, random_state = 213)

In [ ]:
#construct models
#print(X)
reg = LinearRegression().fit(train_x, (train_y))


In [ ]:
#train model on test 
Xtest = test[['session_id', 'music']]

print(np.unique(reg.predict(Xtest)))

In [ ]:
#dummy variables 